<a href="https://colab.research.google.com/github/mohammedaymanulhaque-dev/projects/blob/main/MAJOR_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, request, render_template_string, session, redirect
import random
import matplotlib.pyplot as plt
import base64
from io import BytesIO

app = Flask(__name__)
app.secret_key = "quiz_secret_key"

# ------------------ SAMPLE DATA ------------------

SAMPLE_TEXTS = [
    {"text": "Machine learning allows computers to learn from data without being explicitly programmed."},
    {"text": "Supervised learning uses labeled data to train models."},
    {"text": "Unsupervised learning finds patterns or structures in unlabeled data."},
    {"text": "Reinforcement learning involves training agents through rewards and punishments."},
    {"text": "Neural networks are composed of layers of interconnected nodes called neurons."}
]


# ------------------ HELPERS ------------------

def make_study_plan(subject, hours):
    plan = []
    for i in range(1, hours + 1):
        plan.append({
            "hour": i,
            "topic": f"{subject} topic {i}"
        })
    return plan


def generate_tips_from_text(text):
    return [
        "Break down large topics into smaller, manageable chunks.",
        "Revise your notes after each study session.",
        "Test yourself regularly to identify weak areas.",
        "Take short breaks to retain focus.",
        "Teach others to reinforce your understanding."
    ]


def generate_quiz(subject):
    """Generate quiz questions based on subject name"""
    subject = subject.lower()
    questions = []

    if subject == "machine learning":
        questions = [
            {"question": "1) What is the main purpose of machine learning?",
             "answer": "To enable computers to learn from data and make predictions or decisions."},
            {"question": "2) Which type of machine learning uses labeled training data to make predictions?",
             "answer": "Supervised learning."},
            {"question": "3) Which of the following is an example of a machine learning application?",
             "answer": "Spam email detection."}
        ]
    elif subject == "python":
        questions = [
            {"question": "1) What is the difference between tuples, lists, and sets?",
             "answer": "Tuples are immutable, lists are mutable, and sets are unordered collections of unique items."},
            {"question": "2) What do you mean by Python programming?",
             "answer": "Python is a high-level, interpreted programming language known for readability and versatility."},
            {"question": "3) What are loops and control statements in Python?",
             "answer": "Loops like for and while repeat tasks; control statements like if, break, continue manage flow."}
        ]
    elif subject == "nlp":
        questions = [
            {"question": "1) Explain the difference between syntactic and semantic analysis in NLP.",
             "answer": "Syntactic analysis checks grammar and structure, while semantic analysis focuses on meaning."},
            {"question": "2) What is tokenization?",
             "answer": "The process of splitting text into words, phrases, or symbols called tokens."},
            {"question": "3) What are stopwords, and why do you remove them?",
             "answer": "Stopwords are common words (like 'the', 'is') that are removed to reduce noise in text analysis."}
        ]
    else:
        # Default if unknown subject
        questions = [
            {"question": f"1) What do you know about {subject}?", "answer": f"This is an example answer for {subject}."},
            {"question": f"2) Explain any concept related to {subject}.", "answer": f"A sample explanation for {subject}."},
            {"question": f"3) Why is {subject} important to study?", "answer": f"{subject} helps in understanding advanced topics."}
        ]

    return questions


# ------------------ ROUTES ------------------

@app.route("/")
def home_redirect():
    return "<script>window.location.href='/study'</script>"


@app.route("/study", methods=["GET", "POST"])
def study_plan():
    if request.method == "POST":
        subject = request.form.get("subject")
        hours = request.form.get("hours")

        if not subject or not hours:
            return "Please enter both subject and hours."

        hours = int(hours)
        plan = make_study_plan(subject, hours)
        quiz = generate_quiz(subject)
        tips = generate_tips_from_text(" ".join(t["text"] for t in SAMPLE_TEXTS))

        # Store quiz in session for later
        session["quiz"] = quiz
        session["current"] = 0
        session["score"] = 0
        session["subject"] = subject

        return render_template_string("""
        <!doctype html>
        <title>Study Plan</title>
        <style>
        body {font-family: Arial; margin: 40px; max-width: 800px;}
        .btn {background-color: #1a73e8; color: white; padding: 8px 12px; border: none; border-radius: 5px; cursor: pointer;}
        </style>

        <h2>Your Study Plan</h2>
        <p>📘 Study Plan for <b>{{subject}}</b></p>
        <p>→ Total Hours: {{hours}}</p>
        <ul>
          {% for p in plan %}
            <li>Hour {{loop.index}}: Focus on {{p['topic']}}</li>
          {% endfor %}
        </ul>

        <h3>Study Tips</h3>
        <ul>
        {% for t in tips %}
          <li>{{t}}</li>
        {% endfor %}
        </ul>

        <form method="get" action="/quiz">
          <button class="btn">Start Quiz</button>
        </form>
        """, subject=subject, hours=hours, plan=plan, tips=tips)

    # Initial form
    return render_template_string("""
    <form method="POST" style="font-family: Arial; margin: 40px;">
        <label>Subject:</label><br>
        <input name="subject" placeholder="e.g. Machine Learning, Python, NLP"><br><br>
        <label>Hours:</label><br>
        <input name="hours" type="number"><br><br>
        <button type="submit">Generate Study Plan</button>
    </form>
    """)


@app.route("/quiz", methods=["GET", "POST"])
def quiz():
    quiz = session.get("quiz", [])
    current = session.get("current", 0)
    score = session.get("score", 0)

    if not quiz:
        return "<h3>No quiz found. Please start from the study page.</h3>"

    # When user submits an answer
    if request.method == "POST":
        answer = request.form.get("answer", "").strip().lower()
        correct_answer = quiz[current - 1]["answer"].lower()
        if any(word in answer for word in correct_answer.split()[:3]):
            score += 1
        session["score"] = score

    # Move to next question
    if current >= len(quiz):
        return redirect("/summary")

    question = quiz[current]["question"]
    session["current"] = current + 1

    return render_template_string("""
    <!doctype html>
    <title>Quiz</title>
    <style>
    body {font-family: Arial; margin: 40px; max-width: 700px;}
    .btn {background-color: #28a745; color: white; padding: 8px 12px; border: none; border-radius: 5px; cursor: pointer;}
    </style>

    <h2>Quiz on {{subject}}</h2>
    <form method="POST">
        <p><b>{{question}}</b></p>
        <textarea name="answer" rows="3" cols="60" placeholder="Type your answer here..." required></textarea><br><br>
        <button class="btn">Submit Answer</button>
    </form>
    """, question=question, subject=session.get("subject", "General"))


@app.route("/summary")
def summary():
    quiz = session.get("quiz", [])
    score = session.get("score", 0)
    total_questions = len(quiz)
    marks = score * 2  # 2 marks per correct answer

    # Scatter plot data
    x = list(range(1, total_questions + 1))
    y = [2 if i < score else 0 for i in range(total_questions)]

    plt.figure(figsize=(5, 4))
    plt.scatter(x, y, color='blue')
    plt.xlabel("Question Number")
    plt.ylabel("Marks (2 per correct)")
    plt.title("Quiz Performance Scatter Plot")
    plt.grid(True)

    img = BytesIO()
    plt.savefig(img, format='png')
    img.seek(0)
    plot_data = base64.b64encode(img.getvalue()).decode()

    return render_template_string("""
    <!doctype html>
    <title>Summary</title>
    <style>
    body {font-family: Arial; margin: 40px; text-align: center;}
    img {border: 1px solid #ccc; margin-top: 20px;}
    </style>

    <h2>🎓 Quiz Summary - {{subject}}</h2>
    <p>Total Questions: {{total}}</p>
    <p>Correct Answers: {{score}}</p>
    <p>Total Marks: {{marks}}</p>

    <h3>Performance Scatter Plot</h3>
    <img src="data:image/png;base64,{{plot_data}}" width="500">
    <br><br>
    <a href="/study">🔁 Take Another Quiz</a>
    """, total=total_questions, score=score, marks=marks,
       plot_data=plot_data, subject=session.get("subject", "General"))


# ------------------ RUN APP ------------------

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)
